# 问答基准测试：国情咨文演讲

在这里，我们将讨论如何在国情咨文演讲的问答任务上进行性能基准测试。

强烈建议您启用跟踪来进行任何评估/基准测试。请参见[此处](https://langchain.readthedocs.io/en/latest/tracing.html)了解跟踪是什么以及如何设置它。

In [1]:
# Comment this out if you are NOT using tracing
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

## 加载数据
首先，让我们加载数据。

In [2]:
from langchain.evaluation.loading import load_dataset
dataset = load_dataset("question-answering-state-of-the-union")

Found cached dataset json (/Users/harrisonchase/.cache/huggingface/datasets/LangChainDatasets___json/LangChainDatasets--question-answering-state-of-the-union-a7e5a3b2db4f440d/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

## 设置链
现在，我们需要创建一些流水线来进行问答。其中的第一步是针对所涉及的数据创建一个索引。

In [2]:
from langchain.document_loaders import TextLoader
loader = TextLoader("../../modules/state_of_the_union.txt")

In [3]:
from langchain.indexes import VectorstoreIndexCreator

In [4]:
vectorstore = VectorstoreIndexCreator().from_loaders([loader]).vectorstore

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


现在我们可以创建一个问答链。

In [5]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [6]:
chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=vectorstore.as_retriever(), input_key="question")

## 进行预测

首先，我们可以一次预测一个数据点。以这种粒度进行操作允许我们详细研究输出，并且比运行多个数据点要便宜得多。

In [17]:
chain(dataset[0])

{'question': 'What is the purpose of the NATO Alliance?',
 'answer': 'The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.',
 'result': ' The NATO Alliance was created to secure peace and stability in Europe after World War 2.'}

## 进行多个预测
现在，我们可以进行多个预测。

In [7]:
predictions = chain.apply(dataset)

## 评估性能
现在，我们可以评估预测结果。我们可以首先通过目测它们来查看它们。

In [8]:
predictions[0]

{'question': 'What is the purpose of the NATO Alliance?',
 'answer': 'The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.',
 'result': ' The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.'}

接下来，我们可以使用语言模型对其进行编程评分。

In [9]:
from langchain.evaluation.qa import QAEvalChain

In [10]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(dataset, predictions, question_key="question", prediction_key="result")

我们可以将评分输出添加到“predictions”字典中，然后获取评分计数。

In [11]:
for i, prediction in enumerate(predictions):
    prediction['grade'] = graded_outputs[i]['text']

In [12]:
from collections import Counter
Counter([pred['grade'] for pred in predictions])

Counter({' CORRECT': 7, ' INCORRECT': 4})

我们还可以过滤出错误示例的数据点并查看它们。

In [13]:
incorrect = [pred for pred in predictions if pred['grade'] == " INCORRECT"]

In [14]:
incorrect[0]

{'question': 'What is the U.S. Department of Justice doing to combat the crimes of Russian oligarchs?',
 'answer': 'The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs.',
 'result': ' The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs and is naming a chief prosecutor for pandemic fraud.',
 'grade': ' INCORRECT'}